# Deliverable 7.1: Nonlinear MPC for Landing

Implement and test a nonlinear MPC controller using CasADi for the full 12-state rocket system.

In [ ]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import sys, os
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [ ]:
from src.rocket import Rocket
from src.pos_rocket_vis import *
from LandMPC.nmpc_land import NmpcCtrl
import numpy as np

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

# Rocket setup
Ts  = 1/20
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
rocket.mass = 1.7  # Do not change!!!
rocket.controller_type = 'NmpcCtrl'  # IMPORTANT: Set for proper NMPC constraint checking

# Visualization setup
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1

In [ ]:
# Landing maneuver: from (3, 2, 10, 30°) to (1, 0, 3, 0°)
sim_time = 15  # simulation length in seconds

# Initial state: at (3, 2, 10, 30°)
x0 = np.array([0, 0, 0,  # angular velocities (wx, wy, wz)
               0, 0, np.deg2rad(30),  # angles (alpha, beta, gamma/roll)
               0, 0, 0,  # linear velocities (vx, vy, vz)
               3, 2, 10])  # positions (x, y, z)

# Target state: (1, 0, 3, 0°)
x_ref = np.array([0.]*9 + [1., 0., 3.])  # Target: x=1, y=0, z=3

# NMPC parameters - use longer horizon for large maneuvers
H = 4.0  # horizon time (increased from 2.0 for better long-term planning)

# Create nonlinear MPC controller (it will trim around x_ref internally)
nmpc = NmpcCtrl(rocket, Ts=Ts, H=H, x_ref=x_ref)
print("\nNonlinear MPC initialized")
print(f"  - Horizon: {nmpc.N} steps ({H}s)")
print(f"  - State dimension: {nmpc.nx}")
print(f"  - Input dimension: {nmpc.nu}")
print(f"  - Uses full nonlinear dynamics (no subsystem decomposition)")
print(f"  - Integration: RK4 (more accurate than Euler)")
print(f"  - Tuning: High terminal cost (P=10*Q) for convergence")

# Store trim point for later use
xs, us = nmpc.xs, nmpc.us

## Test 1: Check Open-Loop Trajectory

Before running closed-loop, verify that the optimal open-loop trajectory from the initial state is reasonable.

In [ ]:
# Compute open-loop optimal trajectory
u0, x_ol, u_ol, t_ol = nmpc.get_u(0.0, x0)

print("Open-loop trajectory computed:")
print(f"  - Initial control: {u0}")
print(f"  - Final state: {x_ol[:, -1]}")
print(f"  - Target state: {xs}")

# Plot open-loop trajectory using the standard plotting function
plot_static_states_inputs(t_ol[:-1], x_ol[:,:-1], u_ol, xs)

## Test 2: Closed-Loop Simulation

In [ ]:
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol = rocket.simulate_land(nmpc, sim_time, H, x0)

## Diagnostics: Check NMPC Setup

Before simulating, let's verify the NMPC controller is working correctly.

In [ ]:

u_test, x_test, u_test_traj, t_test = nmpc.get_u(0.0, x0)


In [ ]:
vis.animate(t_cl[:-1], x_cl[:,:-1], u_cl, T_ol=t_ol[...,:-1], X_ol=x_ol, U_ol=u_ol)
plot_static_states_inputs(t_cl[:-1], x_cl[:,:-1], u_cl, xs)

## Test 3: Settling Time Analysis

Verify convergence to the target state by computing settling time based on error tolerances.

In [ ]:
import matplotlib.pyplot as plt

# Define error tolerances
pos_tol = 0.05  # 5 cm
vel_tol = 0.01  # 1 cm/s
angle_tol = np.deg2rad(1)  # 1 degree
ang_vel_tol = np.deg2rad(2)  # 2 deg/s

# Compute errors relative to equilibrium
pos_error = np.linalg.norm(x_cl[9:12, :] - xs[9:12, np.newaxis], axis=0)
vel_error = np.linalg.norm(x_cl[6:9, :] - xs[6:9, np.newaxis], axis=0)
angle_error = np.abs(x_cl[5, :] - xs[5])
ang_vel_error = np.linalg.norm(x_cl[0:3, :] - xs[0:3, np.newaxis], axis=0)

# Determine when system is settled (all errors below tolerance)
settled = (pos_error < pos_tol) & (vel_error < vel_tol) & \
          (angle_error < angle_tol) & (ang_vel_error < ang_vel_tol)

settled_indices = np.where(settled)[0]
settling_time = t_cl[settled_indices[0]] if len(settled_indices) > 0 else sim_time

print(f"Settling time: {settling_time:.2f} seconds")
print(f"Tolerances: pos={pos_tol}m, vel={vel_tol}m/s, angle={np.rad2deg(angle_tol)}°, ang_vel={np.rad2deg(ang_vel_tol)}°/s")

# Create convergence plots
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

# Position error
axs[0, 0].plot(t_cl, pos_error, linewidth=2, color='blue')
axs[0, 0].axhline(y=pos_tol, color='r', linestyle='--', linewidth=2, label='Tolerance')
axs[0, 0].axvline(x=settling_time, color='g', linestyle='--', linewidth=2,
                  label=f'Settling: {settling_time:.2f}s')
axs[0, 0].set_xlabel('Time [s]', fontsize=12)
axs[0, 0].set_ylabel('Position Error [m]', fontsize=12)
axs[0, 0].set_title('Position Convergence', fontsize=13, fontweight='bold')
axs[0, 0].legend()
axs[0, 0].grid(True, alpha=0.3)

# Velocity error
axs[0, 1].plot(t_cl, vel_error, linewidth=2, color='blue')
axs[0, 1].axhline(y=vel_tol, color='r', linestyle='--', linewidth=2, label='Tolerance')
axs[0, 1].axvline(x=settling_time, color='g', linestyle='--', linewidth=2,
                  label=f'Settling: {settling_time:.2f}s')
axs[0, 1].set_xlabel('Time [s]', fontsize=12)
axs[0, 1].set_ylabel('Velocity Error [m/s]', fontsize=12)
axs[0, 1].set_title('Velocity Convergence', fontsize=13, fontweight='bold')
axs[0, 1].legend()
axs[0, 1].grid(True, alpha=0.3)

# Roll angle error
axs[1, 0].plot(t_cl, np.rad2deg(angle_error), linewidth=2, color='blue')
axs[1, 0].axhline(y=np.rad2deg(angle_tol), color='r', linestyle='--', linewidth=2,
                  label='Tolerance')
axs[1, 0].axvline(x=settling_time, color='g', linestyle='--', linewidth=2,
                  label=f'Settling: {settling_time:.2f}s')
axs[1, 0].set_xlabel('Time [s]', fontsize=12)
axs[1, 0].set_ylabel('Roll Error [deg]', fontsize=12)
axs[1, 0].set_title('Roll Angle Convergence', fontsize=13, fontweight='bold')
axs[1, 0].legend()
axs[1, 0].grid(True, alpha=0.3)

# Angular velocity error
axs[1, 1].plot(t_cl, np.rad2deg(ang_vel_error), linewidth=2, color='blue')
axs[1, 1].axhline(y=np.rad2deg(ang_vel_tol), color='r', linestyle='--', linewidth=2,
                  label='Tolerance')
axs[1, 1].axvline(x=settling_time, color='g', linestyle='--', linewidth=2,
                  label=f'Settling: {settling_time:.2f}s')
axs[1, 1].set_xlabel('Time [s]', fontsize=12)
axs[1, 1].set_ylabel('Angular Velocity Error [deg/s]', fontsize=12)
axs[1, 1].set_title('Angular Velocity Convergence', fontsize=13, fontweight='bold')
axs[1, 1].legend()
axs[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Additional Analysis

For controller comparison plots (Linear MPC vs NMPC), see the [compare_controllers.py](compare_controllers.py) file. Copilot as been be used as help for the plots